<a href="https://colab.research.google.com/github/Kaleab1999/Ethiopia-airlines-tweeter-review-sentiment-analysis/blob/main/st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install text-hammer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 38.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import text_hammer as th
import spacy
import string
import re

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [4]:
df= pd.read_csv('/content/drive/MyDrive/dataset/merge-csv.com__6361876ee50a8.csv' , encoding= 'unicode_escape')
df.head()

,created_at,user_id,name,label,tweet,language,link,Column1
0,2022-10-28 23:15:24 UTC,4.386524e+08,AfricanSon®,negative,Ethiopian Airlines (EA) flight ET-3717 was int...,en,https://twitter.com/samsenge/status/1586134568...,NaN
1,2022-10-28 22:53:28 UTC,1.320000e+18,Bargo YOUSIF,negative,"_ details: It's #Ethiopian_Airlines flight, sm...",en,https://twitter.com/BargoYousif/status/1586129...,NaN
2,2022-10-28 22:21:31 UTC,8.226798e+08,Dr. Aklog Birara,positive,I look forward to seeing integrated infrastruc...,en,https://twitter.com/DrAklogBirara/status/15861...,NaN
3,2022-10-28 19:46:19 UTC,1.470000e+18,bill,negative,@jaeyoonieee me getting on ethiopian airlines ...,en,https://twitter.com/ysfkd_/status/158608195096...,NaN
4,2022-10-28 16:40:48 UTC,1.570000e+18,OverMeon,neutral,#ETH552 ET-AVE A350 941 Ethiopian Airlines ADD...,en,https://twitter.com/OverMeon/status/1586035262...,NaN


In [5]:
df.shape

(4429, 8)

In [6]:
y= df.iloc[:, 1]
y.head()

0    4.386524e+08
1    1.320000e+18
2    8.226798e+08
3    1.470000e+18
4    1.570000e+18
Name: user_id, dtype: float64

In [7]:
nlp= spacy.load('en_core_web_sm')

stopwords= list(nlp.Defaults.stop_words)
print(stopwords)

['therefore', 'made', 'than', 'so', 'herein', 'via', 'thereupon', 'either', 'almost', 'per', 'how', 'around', 'did', 'on', 'six', 'themselves', 'whenever', 'between', 'if', 'using', 'both', 'get', 'for', 'being', 'where', 'hence', 'wherein', 'everything', 'formerly', 'often', 'yourself', 'top', 'very', 'one', '’ve', 'whither', 'done', 'whole', 'full', 'too', 'eight', 'move', 'amount', 'which', 'nine', 'cannot', 'just', 'whereupon', 'ever', 'there', '‘ve', 'after', 'whose', 'from', 'who', 'together', 'whatever', 'most', 'you', 'due', 'me', 'with', 'hereupon', 'seem', 'someone', 'amongst', 'him', 'yourselves', 'used', 'i', 'bottom', 'his', 'ours', 'fifteen', 'sometime', 'up', 'was', 'whereafter', "'s", 'in', 'besides', 'show', 'forty', 'still', 'nobody', '‘d', 'every', 'these', "'ll", 'less', 'or', 'is', "'m", 'what', 'onto', 'doing', 'somehow', 'against', 'hundred', 'indeed', 'would', 'elsewhere', 'whereas', 'before', 'because', 'hereby', 'towards', 'also', 'same', 'yet', 'few', 'withou

In [8]:
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

In [9]:
from gensim.parsing.preprocessing import remove_stopwords

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
 import nltk
 nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
def text_preprocessor(x):

  doc = nlp(x)
  
  text= [str(word).lower().strip() for word in doc ]
  text = str(text).lower().replace('\\', '').replace('_', ' ')
  text = th.cont_exp(text)
  text = th.remove_emails(text)
  text = th.remove_urls(text)
  text = th.remove_html_tags(text)
  text = th.remove_rt(text)
  text = th.remove_accented_chars(text)
  text = th.remove_special_chars(text)
  text = re.sub("(.)\\1{2,}", "\\1", text)
  text= re.sub('[^a-zA-Z]', ' ',text)
  text = lemmatizer.lemmatize(text, pos='a')
  #text= text if text not in stopwords else " "
  filtered_sentence = remove_stopwords(text)
  

  return filtered_sentence

In [17]:
df['processed']= df['tweet'].apply(text_preprocessor)
df[['processed','tweet']].head()

,processed,tweet
0,ethiopian airlines ea flight et intercepted fi...,Ethiopian Airlines (EA) flight ET-3717 was int...
1,details s ethiopian airlines flight smoke seen...,"_ details: It's #Ethiopian_Airlines flight, sm..."
2,look forward seeing integrated infrastructure ...,I look forward to seeing integrated infrastruc...
3,jaeyoonie getting ethiopian airlines overrun w...,@jaeyoonieee me getting on ethiopian airlines ...
4,eth et ave ethiopian airlines add dub bole int...,#ETH552 ET-AVE A350 941 Ethiopian Airlines ADD...


In [18]:
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [19]:
df.head()

,created_at,user_id,name,label,tweet,language,link,Column1,processed
0,2022-10-28 23:15:24 UTC,4.386524e+08,AfricanSon®,negative,Ethiopian Airlines (EA) flight ET-3717 was int...,en,https://twitter.com/samsenge/status/1586134568...,NaN,ethiopian airlines ea flight et intercepted fi...
1,2022-10-28 22:53:28 UTC,1.320000e+18,Bargo YOUSIF,negative,"_ details: It's #Ethiopian_Airlines flight, sm...",en,https://twitter.com/BargoYousif/status/1586129...,NaN,details s ethiopian airlines flight smoke seen...
2,2022-10-28 22:21:31 UTC,8.226798e+08,Dr. Aklog Birara,positive,I look forward to seeing integrated infrastruc...,en,https://twitter.com/DrAklogBirara/status/15861...,NaN,look forward seeing integrated infrastructure ...
3,2022-10-28 19:46:19 UTC,1.470000e+18,bill,negative,@jaeyoonieee me getting on ethiopian airlines ...,en,https://twitter.com/ysfkd_/status/158608195096...,NaN,jaeyoonie getting ethiopian airlines overrun w...
4,2022-10-28 16:40:48 UTC,1.570000e+18,OverMeon,neutral,#ETH552 ET-AVE A350 941 Ethiopian Airlines ADD...,en,https://twitter.com/OverMeon/status/1586035262...,NaN,eth et ave ethiopian airlines add dub bole int...


In [20]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 5.5 MB 46.6 MB/s 
     |████████████████████████████████| 1.3 MB 53.5 MB/s 
     |████████████████████████████████| 163 kB 67.6 MB/s 
     |████████████████████████████████| 7.6 MB 61.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5da205efa06c00bbdce86a02427a3c15e6f93cab4960a87647d67d7e76f1973d
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [21]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df['vector']= df['processed'].apply(model.encode)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [22]:
df['processed']= df['processed'].astype('category')

In [23]:
df.columns

Index(['created_at', 'user_id', 'name', 'label', 'tweet', 'language', 'link',
       'Column1', 'processed', 'vector'],
      dtype='object')

In [24]:
df[['processed','vector', 'label']].head()

,processed,vector,label
0,ethiopian airlines ea flight et intercepted fi...,"[-0.038563192, -0.016681375, -0.100923315, 0.0...",negative
1,details s ethiopian airlines flight smoke seen...,"[-0.0024794496, 0.06374694, 0.004985377, 0.038...",negative
2,look forward seeing integrated infrastructure ...,"[-0.002938491, 0.07285521, 0.010665068, 0.0460...",positive
3,jaeyoonie getting ethiopian airlines overrun w...,"[0.04494247, 0.14852953, 0.009378759, 0.067382...",negative
4,eth et ave ethiopian airlines add dub bole int...,"[-0.013125635, 0.029042985, -0.027466798, -0.0...",neutral


In [25]:
import gensim.downloader as api
import numpy as np
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import spacy
import string

In [52]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [27]:
import numpy as np
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [54]:
df['word2vec']= df['processed'].apply(sent_vec)

df[['word2vec','processed', 'vector', 'label']].head()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


,word2vec,processed,vector,label
0,"[-0.19284727515243902, 0.11793431227769309, -0...",ethiopian airlines ea flight et intercepted fi...,"[-0.038563192, -0.016681375, -0.100923315, 0.0...",negative
1,"[-0.18535800242987205, 0.11863215889517717, -0...",details s ethiopian airlines flight smoke seen...,"[-0.0024794496, 0.06374694, 0.004985377, 0.038...",negative
2,"[-0.1983869789290602, 0.10877355867928833, -0....",look forward seeing integrated infrastructure ...,"[-0.002938491, 0.07285521, 0.010665068, 0.0460...",positive
3,"[-0.16255405970982142, 0.09600456393494898, -0...",jaeyoonie getting ethiopian airlines overrun w...,"[0.04494247, 0.14852953, 0.009378759, 0.067382...",negative
4,"[-0.16932586055473994, 0.1156755485790688, -0....",eth et ave ethiopian airlines add dub bole int...,"[-0.013125635, 0.029042985, -0.027466798, -0.0...",neutral


In [29]:
from sentence_transformers import util
cosine= util.pytorch_cos_sim(df['vector'][1], df['vector'][3])

print(cosine)

tensor([[0.4362]])


In [30]:
df['airline_sentiment']= df['label'].astype('category').cat.codes

df['airline_sentiment'].head()

0    0
1    0
2    2
3    0
4    1
Name: airline_sentiment, dtype: int8

In [31]:
X= df['vector'].tolist()
y= df['airline_sentiment'].tolist()

In [32]:
from imblearn.over_sampling import SMOTE

over= SMOTE()
X, y = over.fit_resample(X, y)

In [33]:
len(y)

6822

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_text, y_train, y_test= train_test_split(X, y, test_size=0.2, stratify=y)

In [35]:
len(X_train)

5457

In [36]:
len(y_train)

5457

In [37]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier()

rf.fit(X_train, y_train)

RandomForestClassifier()

In [38]:
pred= rf.predict(X_text)

In [39]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)


[[424  24   7]
 [ 30 379  46]
 [ 10  44 401]]
0.882051282051282
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       455
           1       0.85      0.83      0.84       455
           2       0.88      0.88      0.88       455

    accuracy                           0.88      1365
   macro avg       0.88      0.88      0.88      1365
weighted avg       0.88      0.88      0.88      1365



In [40]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression()

lr.fit(X_train, y_train)

predi= lr.predict(X_text)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,predi)
print(matrix)
score=accuracy_score(y_test,predi)
print(score)
report=classification_report(y_test,predi)
print(report)


[[395  34  26]
 [ 42 359  54]
 [ 34  63 358]]
0.8146520146520146
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       455
           1       0.79      0.79      0.79       455
           2       0.82      0.79      0.80       455

    accuracy                           0.81      1365
   macro avg       0.81      0.81      0.81      1365
weighted avg       0.81      0.81      0.81      1365



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [41]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

pred= gnb.predict(X_text)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[359  45  51]
 [ 55 337  63]
 [ 51  87 317]]
0.7421245421245422
              precision    recall  f1-score   support

           0       0.77      0.79      0.78       455
           1       0.72      0.74      0.73       455
           2       0.74      0.70      0.72       455

    accuracy                           0.74      1365
   macro avg       0.74      0.74      0.74      1365
weighted avg       0.74      0.74      0.74      1365



In [42]:
randomclassifier=RandomForestClassifier(n_estimators=90)
randomclassifier.fit(X_train,y_train)

pred= randomclassifier.predict(X_text)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[423  24   8]
 [ 34 379  42]
 [ 16  51 388]]
0.8717948717948718
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       455
           1       0.83      0.83      0.83       455
           2       0.89      0.85      0.87       455

    accuracy                           0.87      1365
   macro avg       0.87      0.87      0.87      1365
weighted avg       0.87      0.87      0.87      1365



In [43]:
from sklearn.tree import DecisionTreeClassifier
dTree_clf = DecisionTreeClassifier()
dTree_clf.fit(X_train,y_train)

pred= dTree_clf.predict(X_text)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[382  39  34]
 [ 58 314  83]
 [ 28  78 349]]
0.7655677655677655
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       455
           1       0.73      0.69      0.71       455
           2       0.75      0.77      0.76       455

    accuracy                           0.77      1365
   macro avg       0.76      0.77      0.76      1365
weighted avg       0.76      0.77      0.76      1365



In [45]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
predictions = knn.predict(X_text)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)



[[425   1  29]
 [ 70 255 130]
 [ 22  15 418]]
0.8043956043956044
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       455
           1       0.94      0.56      0.70       455
           2       0.72      0.92      0.81       455

    accuracy                           0.80      1365
   macro avg       0.83      0.80      0.80      1365
weighted avg       0.83      0.80      0.80      1365



In [46]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import numpy as np
x_t= np.array(X_train)
y_t= np.array(y_train)
x= np.array(X_text)
y= np.array(y_test)

In [48]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(x_t,y_t)

predictions = xgb_classifier.predict(x)

matrix=confusion_matrix(y,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

[[406  33  16]
 [ 45 356  54]
 [ 24  56 375]]
0.832967032967033
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       455
           1       0.80      0.78      0.79       455
           2       0.84      0.82      0.83       455

    accuracy                           0.83      1365
   macro avg       0.83      0.83      0.83      1365
weighted avg       0.83      0.83      0.83      1365



In [49]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC()  
classifier.fit(x_t, y_t)  
pred= classifier.predict(x)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y,pred)
print(matrix)
score=accuracy_score(y,pred)
print(score)
report=classification_report(y,pred)
print(report)

[[422  19  14]
 [ 35 376  44]
 [ 17  46 392]]
0.8717948717948718
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       455
           1       0.85      0.83      0.84       455
           2       0.87      0.86      0.87       455

    accuracy                           0.87      1365
   macro avg       0.87      0.87      0.87      1365
weighted avg       0.87      0.87      0.87      1365



In [55]:
y= df['airline_sentiment'].tolist()
X= df['word2vec'].tolist()

In [56]:
from imblearn.over_sampling import SMOTE

over= SMOTE()

X, y= over.fit_resample(X,y)

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [58]:
len(y_train)

5457

In [59]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC()  
classifier.fit(X_train, y_train)  

pred= classifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[314  69  72]
 [122 209 124]
 [139 106 210]]
0.536996336996337
              precision    recall  f1-score   support

           0       0.55      0.69      0.61       455
           1       0.54      0.46      0.50       455
           2       0.52      0.46      0.49       455

    accuracy                           0.54      1365
   macro avg       0.54      0.54      0.53      1365
weighted avg       0.54      0.54      0.53      1365



In [60]:
from sklearn.ensemble import RandomForestClassifier
randomclassifier=RandomForestClassifier(n_estimators=90, criterion='gini')
randomclassifier.fit(X_train,y_train)

pred= randomclassifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[398  33  24]
 [ 31 357  67]
 [ 20  53 382]]
0.832967032967033
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       455
           1       0.81      0.78      0.80       455
           2       0.81      0.84      0.82       455

    accuracy                           0.83      1365
   macro avg       0.83      0.83      0.83      1365
weighted avg       0.83      0.83      0.83      1365



In [65]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)



[[404  17  34]
 [129 192 134]
 [ 83  30 342]]
0.6871794871794872
              precision    recall  f1-score   support

           0       0.66      0.89      0.75       455
           1       0.80      0.42      0.55       455
           2       0.67      0.75      0.71       455

    accuracy                           0.69      1365
   macro avg       0.71      0.69      0.67      1365
weighted avg       0.71      0.69      0.67      1365



In [62]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression()

lr.fit(X_train, y_train)

predi= lr.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,predi)
print(matrix)
score=accuracy_score(y_test,predi)
print(score)
report=classification_report(y_test,predi)
print(report)

[[310  76  69]
 [103 239 113]
 [132 117 206]]
0.5531135531135531
              precision    recall  f1-score   support

           0       0.57      0.68      0.62       455
           1       0.55      0.53      0.54       455
           2       0.53      0.45      0.49       455

    accuracy                           0.55      1365
   macro avg       0.55      0.55      0.55      1365
weighted avg       0.55      0.55      0.55      1365



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [66]:
from sklearn.tree import DecisionTreeClassifier
dTree_clf = DecisionTreeClassifier()
dTree_clf.fit(X_train,y_train)

pred= dTree_clf.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[352  62  41]
 [ 68 276 111]
 [ 60  92 303]]
0.6820512820512821
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       455
           1       0.64      0.61      0.62       455
           2       0.67      0.67      0.67       455

    accuracy                           0.68      1365
   macro avg       0.68      0.68      0.68      1365
weighted avg       0.68      0.68      0.68      1365



In [70]:
from sklearn.tree import DecisionTreeClassifier
dTree_clf = DecisionTreeClassifier()
dTree_clf.fit(X_train,y_train)

pred= dTree_clf.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[340  64  51]
 [ 64 280 111]
 [ 67  95 293]]
0.6688644688644688
              precision    recall  f1-score   support

           0       0.72      0.75      0.73       455
           1       0.64      0.62      0.63       455
           2       0.64      0.64      0.64       455

    accuracy                           0.67      1365
   macro avg       0.67      0.67      0.67      1365
weighted avg       0.67      0.67      0.67      1365



In [68]:
import numpy as np
x_t= np.array(X_train)
y_t= np.array(y_train)
x= np.array(X_test)
y= np.array(y_test)

In [69]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(x_t,y_t)

predictions = xgb_classifier.predict(x)

matrix=confusion_matrix(y,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

[[348  54  53]
 [ 61 293 101]
 [ 83  82 290]]
0.6820512820512821
              precision    recall  f1-score   support

           0       0.71      0.76      0.73       455
           1       0.68      0.64      0.66       455
           2       0.65      0.64      0.65       455

    accuracy                           0.68      1365
   macro avg       0.68      0.68      0.68      1365
weighted avg       0.68      0.68      0.68      1365



In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 70, stop = 120, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
criterion=['entropy','gini']
# Minimum number of samples required to split a node
#min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
#bootstrap = [True, False]

In [ ]:
# Create the param grid
'''param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'criterion':criterion
               }
print(param_grid)'''

In [ ]:
#rf_Model = RandomForestClassifier()

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#rf_RandomGrid = RandomizedSearchCV(estimator = rf_Model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)

In [ ]:
#rf_RandomGrid.fit(X_train, y_train)

In [ ]:
#rf_RandomGrid.best_params_

In [ ]:
#print (f'Train Accuracy - : {rf_RandomGrid.score(X_train,y_train):.3f}')
#print (f'Test Accuracy - : {rf_RandomGrid.score(X_text,y_test):.3f}')

In [ ]:
'''rfe = RandomForestClassifier(n_estimators= 120,min_samples_split= 5, min_samples_leaf = 2,max_features= 'sqrt',max_depth= 4,bootstrap= False)

rfe.fit(X_train, y_train)

pred= rfe.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)'''